---


---


This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [134]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

pre_train_df = pd.read_csv('train.csv', encoding = 'ISO-8859-1' )
test_df = pd.read_csv('test.csv', encoding = 'ISO-8859-1' )
address_df = pd.read_csv('addresses.csv')
latlons_df = pd.read_csv('latlons.csv')

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [135]:
pre_train_df.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [140]:
address_df.head(10)

In [141]:
latlons_df.head(10)

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295
5,"11751 englewood, Detroit MI",42.243928,-83.204712
6,"6442 heyden, Detroit MI",42.337779,-83.237171
7,"15703 kentfield, Detroit MI 48223",42.404354,-83.240920
8,"3818 clements, Detroit MI",42.389808,-83.133463
9,"431 congress, Detroit MI 48226",42.332344,-83.041562


# Investigating potential for data leakage
Initial look at what is in some of the suspect data columns for data leakage. We have the target label (compliance), and also a few features which violate the "No Time Machine Condition". 

In [153]:
print(pre_train_df['payment_status'].unique())
print(pre_train_df['collection_status'].unique())
print(pre_train_df['compliance_detail'].unique())
print(pre_train_df['compliance'].unique()) # this is the target label


['NO PAYMENT APPLIED' 'PAID IN FULL' 'PARTIAL PAYMENT APPLIED']
[nan 'IN COLLECTION']
['non-compliant by no payment' 'compliant by late payment within 1 month'
 'not responsible by disposition'
 'not responsible by pending judgment disposition'
 'non-compliant by late payment more than 1 month'
 'compliant by early payment' 'compliant by on-time payment'
 'compliant by payment with no scheduled hearing'
 'compliant by payment on unknown date' 'compliant by no fine']
[  0.   1.  nan]


 Definitely need to remove payment date, payment_amount to avoid data leakage. Balance due often 0 when payment date is not NaN, however it appears that sometimes the balance is not paid in full. A partial balance paid also maps to a positive label for compliance. This information would not be available in a real life prediction scenario/production so it should not be included for our learning algorithm. 

# Some exploratory data analysis
Lets get an idea of the features and decide which ones to retain for training the learning algorithm, and if any other features could be derived from the existing set. 

In [154]:
pre_train_df[['balance_due', 'ticket_issued_date', 'hearing_date', 'payment_date', 'payment_amount', 'compliance', 'compliance_detail']].sample(20)

,balance_due,ticket_issued_date,hearing_date,payment_date,payment_amount,compliance,compliance_detail
219520,305.0,2010-06-21 13:00:00,2010-11-04 15:00:00,NaN,0.0,0.0,non-compliant by no payment
204259,305.0,2009-11-12 09:00:00,2010-05-07 13:30:00,NaN,0.0,0.0,non-compliant by no payment
76777,140.0,2006-11-17 13:00:00,2007-02-07 13:30:00,NaN,0.0,0.0,non-compliant by no payment
77638,25.0,2006-11-28 10:40:00,2006-12-14 10:30:00,2007-01-09 00:00:00,280.0,1.0,compliant by late payment within 1 month
205642,0.0,2009-12-03 10:15:00,2010-03-22 15:00:00,NaN,0.0,NaN,not responsible by disposition
186873,305.0,2009-05-15 09:10:00,2009-09-08 09:00:00,NaN,0.0,0.0,non-compliant by no payment
127067,305.0,2008-01-23 08:50:00,2008-02-21 15:00:00,NaN,0.0,0.0,non-compliant by no payment
144245,305.0,2008-05-13 09:15:00,2008-06-20 10:30:00,NaN,0.0,0.0,non-compliant by no payment
170257,305.0,2008-11-26 11:00:00,2009-04-06 15:00:00,NaN,0.0,0.0,non-compliant by no payment
143504,305.0,2008-05-07 09:00:00,2008-06-16 13:30:00,NaN,0.0,0.0,non-compliant by no payment


Here we can gain a bit more insight into the data. We want to predict if the ticket will be paid *on time*, so we can see that samples with a balance of 0, but presumably a very late payment are labeled non-compliant. Payments that are made within 1 month of a 

In [155]:
pre_train_df[['ticket_issued_date','hearing_date', 'compliance_detail']].sample(20)

,ticket_issued_date,hearing_date,compliance_detail
151919,2008-07-17 09:00:00,2008-08-11 09:00:00,non-compliant by no payment
71863,2006-10-11 11:00:00,2006-11-02 09:00:00,non-compliant by late payment more than 1 month
110196,2007-09-11 09:35:00,2008-02-21 09:00:00,not responsible by disposition
190573,2009-06-24 09:50:00,2009-10-15 09:00:00,non-compliant by no payment
245779,2011-09-09 22:15:00,2011-09-30 09:00:00,non-compliant by no payment
224134,2010-08-13 09:00:00,2011-04-08 13:30:00,non-compliant by no payment
196431,2009-08-26 11:20:00,2009-09-17 10:30:00,non-compliant by late payment more than 1 month
222260,2010-07-22 09:00:00,NaN,not responsible by disposition
243833,2011-07-25 09:35:00,2011-08-17 09:00:00,non-compliant by no payment
69607,2006-09-27 10:45:00,2006-10-26 09:00:00,non-compliant by no payment


In [156]:
pre_train_df.sample(4)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
122538,149212,"Buildings, Safety Engineering & Env Department","Devaney, John","SULL, ROBERT",440.0,PETERBORO,NaN,137.0,TOPEKA DR,NORTHRIDGE,...,0.0,580.0,0.0,580.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0.0
46221,68939,Department of Public Works,"Moore, David","OJINAKA, MARTIN C.",1605.0,GLYNN CT,NaN,1685.0,GLYNN CT.,DETROIT,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0.0
237868,273034,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","LLC, COLASSAL",17504.0,LIVERNOIS,NaN,4269.0,WOODWARD #225,BLOOMFIELD,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
1879,20567,"Buildings, Safety Engineering & Env Department","Sharpe, Anthony","GADD, DONALD",6817.0,GRANDVILLE,NaN,25265.0,LYNDON,REDFORD,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0.0


In [157]:
address_df.head(5)

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


The lat/longs may be correlated with compliance, so lets add this to our training dataframe by doing some merges.
The addresses are prone to errors and don't really carry any additional information than the lat/longs, so it is unneccessary to include.

In [158]:

locations_df = pd.merge(latlons_df,
                 address_df[['ticket_id', 'address']],
                 on='address')

pre_train2 = pd.merge(pre_train_df,
                 locations_df[['ticket_id', 'lat', 'lon']],
                 on='ticket_id')

test_df =  pd.merge(test_df,
                 locations_df[['ticket_id', 'lat', 'lon']],
                 on='ticket_id')


In [11]:
test_df.head(10)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.407581,-82.986642
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.426239,-83.238259
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.426239,-83.238259
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.309661,-83.122426
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.308830,-83.121116
5,285345,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.308830,-83.121116
6,285347,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,50.0,20.0,10.0,5.0,0.0,0.0,85.0,NaN,42.308830,-83.121116
7,285342,Department of Public Works,"Talbert, Reginald","NICKOLA CORPORATION, W & H",1605.0,LIVERNOIS,NaN,1382,WHITEHOUSE CT,ROCHESTER HILLS,...,200.0,20.0,10.0,0.0,0.0,0.0,230.0,NaN,42.313314,-83.108636
8,285530,Department of Public Works,"Buchanan, Daryl","INTERSTATE INVESTMENT GROUP LL, .",3408.0,BEATRICE,NaN,341,HAMPTON,GILBERT,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.261245,-83.160878
9,284989,Department of Public Works,"Buchanan, Daryl","YAMAN, BATURAY",8040.0,SARENA,NaN,43494,ELLSWORTH # 20,FREMONT,...,500.0,20.0,10.0,50.0,0.0,0.0,580.0,NaN,42.342537,-83.148025


In [12]:
plt.scatter(pre_train2['lat'], pre_train2['lon'])


In [159]:
pre_train2['grafitti_status'].unique()

array([nan, 'GRAFFITI TICKET'], dtype=object)


Below I am trying to consider whether it is worth it to use violation code, and should be converted to categorical

In [160]:
pre_train2['violation_code'].value_counts()

9-1-36(a)               99091
9-1-81(a)               43471
22-2-88                 28720
9-1-104                 22536
22-2-88(b)               7238
22-2-45                  5394
9-1-43(a) - (Dwellin     5332
9-1-105                  5072
9-1-110(a)               4814
22-2-22                  3755
9-1-103(C)               3537
19450901                 2963
22-2-43                  2903
22-2-17                  1940
22-2-61                  1456
9-1-82(d) - (Dwellin      895
22-2-83(a)(b)(c)          849
61-81.0100/32.0066        812
9-1-43(a) - (Structu      765
22-2-83                   633
9-1-43(a) - (Stories      457
61-5-21                   426
9-1-206                   344
22-2-88(a)                342
9-1-209                   335
9-1-101                   294
22-2-21(b)                286
61-5-18                   273
9-1-107                   222
9-1-111                   201
                        ...  
61-45.0000                  1
9-1-354                     1
9-1-502   

In [15]:
len(pre_train2['violation_code'].unique())

235

In [16]:
pre_train2[['violation_code', 'fine_amount','compliance']]

,violation_code,fine_amount,compliance
0,9-1-36(a),250.0,0.0
1,61-63.0600,750.0,1.0
2,9-1-36(a),250.0,NaN
3,9-1-36(a),250.0,NaN
4,9-1-36(a),250.0,NaN
5,9-1-36(a),250.0,0.0
6,61-63.0500,750.0,0.0
7,61-63.0100,100.0,0.0
8,61-63.0100,100.0,0.0
9,61-63.0600,750.0,0.0


In [161]:
pre_train2[['late_fee', 'compliance', 'compliance_detail']]

,late_fee,compliance,compliance_detail
0,25.0,0.0,non-compliant by no payment
1,75.0,1.0,compliant by late payment within 1 month
2,0.0,NaN,not responsible by disposition
3,0.0,NaN,not responsible by disposition
4,0.0,NaN,not responsible by disposition
5,25.0,0.0,non-compliant by no payment
6,75.0,0.0,non-compliant by no payment
7,10.0,0.0,non-compliant by no payment
8,10.0,0.0,non-compliant by no payment
9,75.0,0.0,non-compliant by no payment


In [162]:
pre_train2['disposition'].unique()

array(['Responsible by Default', 'Responsible by Determination',
       'Not responsible by Dismissal', 'Not responsible by City Dismissal',
       'PENDING JUDGMENT', 'Responsible by Admission',
       'Not responsible by Determination', 'SET-ASIDE (PENDING JUDGMENT)',
       'Responsible (Fine Waived) by Deter'], dtype=object)

# Now, lets start cleaning up the data.

# 1. Convert categorical
Of course, we will need to perform all the same conversions for the test set as well.
For starters, we can convert agency_name, Inspector Name, City, graffiti status, violation_code, disposition


In [163]:
ucats_an = set(pre_train2['agency_name'])|{'<unknown>'}

In [20]:
pre_train2['agency_name']= pd.Categorical(pre_train2['agency_name'],categories=ucats_an).fillna('<unknown>').codes

In [164]:
ucats_an = set(test_df['agency_name'])|{'<unknown>'}
test_df['agency_name']= pd.Categorical(test_df['agency_name'],categories=ucats_an).fillna('<unknown>').codes

In [165]:
# perform the same on inspector name (in reality I'm not sure we'll use all these features, but will jsut leave it here 
# for demonstration)

ucats_an = set(pre_train2['inspector_name'])|{'<unknown>'}
pre_train2['inspector_name']= pd.Categorical(pre_train2['inspector_name'],categories=ucats_an).fillna('<unknown>').codes

ucats_an = set(test_df['inspector_name'])|{'<unknown>'}
test_df['inspector_name']= pd.Categorical(test_df['inspector_name'],categories=ucats_an).fillna('<unknown>').codes

In [166]:
ucats_an = set(pre_train2['violation_code'])|{'<unknown>'}
pre_train2['violation_code']= pd.Categorical(pre_train2['violation_code'],categories=ucats_an).fillna('<unknown>').codes

ucats_an = set(test_df['violation_code'])|{'<unknown>'}
test_df['violation_code']= pd.Categorical(test_df['violation_code'],categories=ucats_an).fillna('<unknown>').codes

In [167]:

ucats_an = set(pre_train2['violation_description'])|{'<unknown>'}
pre_train2['violation_description']= pd.Categorical(pre_train2['violation_description'],categories=ucats_an).fillna('<unknown>').codes
test_df['violation_description']= pd.Categorical(test_df['violation_description'],categories=ucats_an).fillna('<unknown>').codes


ucats_an = set(pre_train2['disposition'])|{'<unknown>'}
pre_train2['disposition']= pd.Categorical(pre_train2['disposition'],categories=ucats_an).fillna('<unknown>').codes
test_df['disposition']= pd.Categorical(test_df['disposition'],categories=ucats_an).fillna('<unknown>').codes


# Derived features

number of days from the ticket issue date to the hearing (if there is one)

In [168]:
from datetime import datetime

def time_diff_days(ticket_issued_date, hearing_date):
    if not hearing_date or type(hearing_date)!=str : return None
    t1 = datetime.strptime(ticket_issued_date, "%Y-%m-%d %H:%M:%S") 
    t2 = datetime.strptime(hearing_date, "%Y-%m-%d %H:%M:%S")
    return abs((t2 - t1).days)    


#print(pre_train2['ticket_issued_date'][3], pre_train2['hearing_date'][3])

#time_diff_days(pre_train2['ticket_issued_date'][5], pre_train2['hearing_date'][5])

pre_train2['time_between_issue_hearing'] = pre_train2.apply(lambda row: time_diff_days(row['ticket_issued_date'],row['hearing_date']), axis=1)

                                 




In [169]:

test_df['time_between_issue_hearing'] = test_df.apply(lambda row: time_diff_days(row['ticket_issued_date'],row['hearing_date']), axis=1)


In [170]:
pre_train2['time_between_issue_hearing']

0          369.0
1          378.0
2          336.0
3            NaN
4          336.0
5          323.0
6          253.0
7          251.0
8          251.0
9          323.0
10         222.0
11         221.0
12         209.0
13         201.0
14         189.0
15         186.0
16         183.0
17         266.0
18         138.0
19         138.0
20         138.0
21         190.0
22         189.0
23         215.0
24         100.0
25          96.0
26          40.0
27          50.0
28          24.0
29          50.0
           ...  
250276      29.0
250277      29.0
250278      37.0
250279      29.0
250280      29.0
250281      30.0
250282      30.0
250283       9.0
250284      25.0
250285      49.0
250286      49.0
250287      10.0
250288      26.0
250289      10.0
250290      40.0
250291      49.0
250292       5.0
250293       5.0
250294      25.0
250295      17.0
250296    3722.0
250297    3722.0
250298    1495.0
250299    1495.0
250300    1495.0
250301    1495.0
250302    1495.0
250303    1495

# Data pruning
We've converted some features, computed one derived quantity. Its not a terrible amount of cleaning, but lets get ready to try out a random forest model to see how well our model might work to predict if a ticket will be paid on time. There are a number of features that will need to be removed from the dataframes- either because they are suspect for data leakage or they might not be very useful in our model prediction. 

The first thing we need to remove is those entries where the target label compliance is NaN. "Note: All tickets where the violators were found not responsible are not considered during evaluation. Thus, these should be dropped. 


In [28]:
pre_train2['compliance'].isnull().sum()

90426

In [29]:
pre_train2 = pre_train2[np.isfinite(pre_train2['compliance'])]

Next lets drop columns that are indicative of leak, or not useful.

In [95]:
# #note that the leaky features indicated are only present in the training data

leaky_features = ['balance_due', 
                  'collection_status',
                  'compliance', 
                  'compliance_detail',
                  'payment_date',
                  'payment_amount',
                  'payment_status']

# it also seems like late fee would be a leak?

not_useful = ['country',
              'city',
              'zip_code',
              'violation_street_name',
              'state',
              'admin_fee',
              'state_fee', 
              'late_fee',
              'clean_up_cost',
              'non_us_str_code',
              'ticket_issued_date',
              'hearing_date',
              'discount_amount',
              'violation_zip_code',
              'grafitti_status',
             'violator_name',
             'inspector_name',
             'violation_street_number',
             'mailing_address_str_number',
             'mailing_address_str_name']








In [96]:
train_features = pre_train2.drop(leaky_features + not_useful, axis=1)

In [97]:
train_features.head(5)

,ticket_id,agency_name,violation_code,violation_description,disposition,fine_amount,judgment_amount,lat,lon,time_between_issue_hearing
0,22056,4,84,224,7,250.0,305.0,42.390729,-83.124268,369.0
1,27586,4,49,123,4,750.0,855.0,42.326937,-83.135118,378.0
5,22046,4,84,224,7,250.0,305.0,42.145257,-83.208233,323.0
6,18738,4,198,164,7,750.0,855.0,42.433466,-83.023493,253.0
7,18735,4,91,90,7,100.0,140.0,42.388641,-83.037858,251.0


In [98]:
test_features = test_df.drop(not_useful, axis=1)

In [99]:
test_features.columns

Index(['ticket_id', 'agency_name', 'violation_code', 'violation_description',
       'disposition', 'fine_amount', 'judgment_amount', 'lat', 'lon',
       'time_between_issue_hearing'],
      dtype='object')

In [100]:
train_features.columns

Index(['ticket_id', 'agency_name', 'violation_code', 'violation_description',
       'disposition', 'fine_amount', 'judgment_amount', 'lat', 'lon',
       'time_between_issue_hearing'],
      dtype='object')

# Taking care of NaN values in the rest of the data
Let us examine how many nulls we have to decide what will be the best way to deal with them.

In [125]:
train_features.isnull().sum()

ticket_id                     0
agency_name                   0
violation_code                0
violation_description         0
disposition                   0
fine_amount                   0
judgment_amount               0
lat                           0
lon                           0
time_between_issue_hearing    0
dtype: int64

In [126]:
test_features.isnull().sum()

agency_name                   0
violation_code                0
violation_description         0
disposition                   0
fine_amount                   0
judgment_amount               0
lat                           0
lon                           0
time_between_issue_hearing    0
compliance                    0
dtype: int64

In [103]:
train_features.lat.fillna(method='pad', inplace=True)
train_features.lon.fillna(method='pad', inplace=True)
train_features.time_between_issue_hearing.fillna(method='pad', inplace=True)

In [104]:
test_features.lat.fillna(method='pad', inplace=True)
test_features.lon.fillna(method='pad', inplace=True)
test_features.time_between_issue_hearing.fillna(method='pad', inplace=True)

In [127]:
test_features.isnull().sum()
train_features.isnull().sum()

ticket_id                     0
agency_name                   0
violation_code                0
violation_description         0
disposition                   0
fine_amount                   0
judgment_amount               0
lat                           0
lon                           0
time_between_issue_hearing    0
dtype: int64

# TODO Use Tree-based feature selection


In [128]:
compliance = pre_train2['compliance']

In [129]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_features,compliance,
                                                   random_state = 0)


In [130]:
X_train.sample(5)

,ticket_id,agency_name,violation_code,violation_description,disposition,fine_amount,judgment_amount,lat,lon,time_between_issue_hearing
144942,174073,4,0,70,7,250.0,305.0,42.403158,-83.146829,58.0
208225,241258,4,84,224,7,250.0,305.0,42.397037,-83.153878,121.0
228931,263060,1,206,236,7,50.0,85.0,42.416539,-83.145316,23.0
8243,28325,4,68,207,2,50.0,80.0,42.439321,-83.141401,140.0
48419,71348,4,84,224,7,250.0,305.0,42.419490,-82.991294,18.0


In [131]:
X_test.sample(5)

,ticket_id,agency_name,violation_code,violation_description,disposition,fine_amount,judgment_amount,lat,lon,time_between_issue_hearing
94339,120354,5,174,4,7,1000.0,1130.0,42.339526,-83.139687,103.0
125273,152328,4,0,70,7,250.0,305.0,42.430785,-82.982725,50.0
12178,32602,4,84,224,7,250.0,305.0,42.444867,-83.093657,87.0
167135,196616,5,145,161,7,50.0,85.0,42.410855,-83.046409,161.0
46712,69521,4,84,224,7,250.0,305.0,42.439345,-83.100881,18.0


In [132]:
X_test.dtypes

ticket_id                       int64
agency_name                      int8
violation_code                  int16
violation_description           int16
disposition                      int8
fine_amount                   float64
judgment_amount               float64
lat                           float64
lon                           float64
time_between_issue_hearing    float64
dtype: object

In [133]:
rf = RandomForestClassifier(n_estimators = 100,
                           random_state=0).fit(X_train, y_train)

In [145]:

print('Accuracy of RF classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.2f}'
     .format(rf.score(X_test, y_test)))

In [113]:
len(rf.predict_proba(X_test))

39970

In [115]:
test_features = test_features.drop('compliance', axis=1)
probabilities_rf = rf.predict_proba(test_features)



In [116]:
probabilities_rf

array([[ 0.68,  0.32],
       [ 0.94,  0.06],
       [ 0.75,  0.25],
       ..., 
       [ 0.67,  0.33],
       [ 0.67,  0.33],
       [ 0.54,  0.46]])

In [117]:
probabilities_rf

array([ 0.68,  0.94,  0.75, ...,  0.67,  0.67,  0.54])

In [118]:
# Note probability is P(0), P(1)
test_features['compliance'] = pd.Series(probabilities_rf[:,1])
test_features.set_index('ticket_id', inplace=True)

In [119]:
test_features.compliance

ticket_id
284932    0.68
285362    0.94
285361    0.75
285338    0.74
285346    0.62
285345    0.79
285347    0.86
285342    0.23
285530    0.85
284989    0.89
285344    0.71
285343    0.84
285340    0.81
285341    0.73
285349    0.64
285348    0.79
284991    0.89
285532    0.89
285406    0.85
285001    0.85
285006    0.84
285405    0.93
285337    0.86
285496    0.66
285497    0.64
285378    0.94
285589    0.89
285585    0.76
285501    0.65
285581    0.96
          ... 
376367    0.90
376366    0.78
376362    0.61
376363    0.60
376365    0.90
376364    0.78
376228    0.70
376265    0.72
376286    0.70
376320    0.74
376314    0.70
376327    0.66
376385    0.74
376435    0.69
376370    0.24
376434    0.77
376459    0.60
376478    0.94
376473    0.58
376484    0.88
376482    0.91
376480    0.96
376479    0.96
376481    0.96
376483    0.87
376496    0.98
376497    0.98
376499    0.67
376500    0.67
369851    0.54
Name: compliance, dtype: float64

# Logistic Regression

In [146]:
#test_features.isnull().sum()

In [147]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

lr = LogisticRegression().fit(X_train, y_train)
clf = lr.fit(X_train, y_train)
y_score_lr = clf.decision_function(X_test)
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)

roc_auc_lr 

In [148]:
clf.predict_proba(test_features)

In [149]:
pd.Series(clf.predict_proba(test_features), index=test_features['ticket_id'])

In [150]:
test_df.set_index('ticket_id', inplace=True)

In [151]:
#TODO: logistic regression, gradient boosted classifier, random forest, MLPClassifier

In [152]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Read data
    pre_train_df = pd.read_csv('train.csv', encoding = 'ISO-8859-1' )
    test_df = pd.read_csv('test.csv', encoding = 'ISO-8859-1' )
    address_df = pd.read_csv('addresses.csv')
    latlons_df = pd.read_csv('latlons.csv')

    #Merge dataframes
    locations_df = pd.merge(latlons_df,
                 address_df[['ticket_id', 'address']],
                 on='address')
    pre_train2 = pd.merge(pre_train_df,
                     locations_df[['ticket_id', 'lat', 'lon']],
                     on='ticket_id')
    test_df =  pd.merge(test_df,
                     locations_df[['ticket_id', 'lat', 'lon']],
                     on='ticket_id')

    # Convert categorical data
    ucats_an = set(pre_train2['agency_name'])|{'<unknown>'}
    pre_train2['agency_name']= pd.Categorical(pre_train2['agency_name'],categories=ucats_an).fillna('<unknown>').codes
    test_df['agency_name']= pd.Categorical(test_df['agency_name'],categories=ucats_an).fillna('<unknown>').codes


    ucats_an = set(pre_train2['inspector_name'])|{'<unknown>'}
    pre_train2['inspector_name']= pd.Categorical(pre_train2['inspector_name'],categories=ucats_an).fillna('<unknown>').codes
    test_df['inspector_name']= pd.Categorical(test_df['inspector_name'],categories=ucats_an).fillna('<unknown>').codes

    ucats_an = set(pre_train2['violation_code'])|{'<unknown>'}
    pre_train2['violation_code']= pd.Categorical(pre_train2['violation_code'],categories=ucats_an).fillna('<unknown>').codes
    ucats_an = set(test_df['violation_code'])|{'<unknown>'}
    test_df['violation_code']= pd.Categorical(test_df['violation_code'],categories=ucats_an).fillna('<unknown>').codes

    ucats_an = set(pre_train2['violation_description'])|{'<unknown>'}
    pre_train2['violation_description']= pd.Categorical(pre_train2['violation_description'],categories=ucats_an).fillna('<unknown>').codes
    ucats_an = set(test_df['violation_description'])|{'<unknown>'}
    test_df['violation_description']= pd.Categorical(test_df['violation_description'],categories=ucats_an).fillna('<unknown>').codes

    ucats_an = set(pre_train2['disposition'])|{'<unknown>'}
    pre_train2['disposition']= pd.Categorical(pre_train2['disposition'],categories=ucats_an).fillna('<unknown>').codes
    ucats_an = set(test_df['disposition'])|{'<unknown>'}
    test_df['disposition']= pd.Categorical(test_df['disposition'],categories=ucats_an).fillna('<unknown>').codes

    #Derived features
    from datetime import datetime
    def time_diff_days(ticket_issued_date, hearing_date):
        if not hearing_date or type(hearing_date)!=str : return None
        t1 = datetime.strptime(ticket_issued_date, "%Y-%m-%d %H:%M:%S") 
        t2 = datetime.strptime(hearing_date, "%Y-%m-%d %H:%M:%S")
        return abs((t2 - t1).days)    

    pre_train2['time_between_issue_hearing'] = pre_train2.apply(lambda row: time_diff_days(row['ticket_issued_date'],row['hearing_date']), axis=1)
    test_df['time_between_issue_hearing'] = test_df.apply(lambda row: time_diff_days(row['ticket_issued_date'],row['hearing_date']), axis=1)

    # Data pruning
    # Remove NaN compliance entries
    pre_train2 = pre_train2[np.isfinite(pre_train2['compliance'])]

    #note that the leaky features indicated are only present in the training data

    leaky_features = ['balance_due', 
                      'collection_status',
                      'compliance', 
                      'compliance_detail',
                      'payment_date',
                      'payment_amount',
                      'payment_status']

    # it also seems like late fee would be a leak?

    not_useful = ['country',
              'city',
              'zip_code',
              'violation_street_name',
              'state',
              'admin_fee',
              'state_fee', 
              'late_fee',
              'clean_up_cost',
              'non_us_str_code',
              'ticket_issued_date',
              'hearing_date',
              'discount_amount',
              'violation_zip_code',
              'grafitti_status',
             'violator_name',
             'inspector_name',
             'violation_street_number',
             'mailing_address_str_number',
             'mailing_address_str_name']



    train_features = pre_train2.drop(leaky_features + not_useful, axis=1)
    test_features = test_df.drop(not_useful, axis=1)

    train_features.lat.fillna(method='pad', inplace=True)
    train_features.lon.fillna(method='pad', inplace=True)
    train_features.time_between_issue_hearing.fillna(method='pad', inplace=True)

    test_features.lat.fillna(method='pad', inplace=True)
    test_features.lon.fillna(method='pad', inplace=True)
    test_features.time_between_issue_hearing.fillna(method='pad', inplace=True)

    compliance = pre_train2['compliance']

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(train_features,
                                                   compliance,
                                                   random_state = 0)

    # Random Forest- Gives AUC of 0.724833898685
    # rf = RandomForestClassifier(n_estimators = 100, random_state=0).fit(X_train, y_train)
    # probabilities_rf = rf.predict_proba(test_features)

    #test_features['compliance'] = pd.Series(probabilities_rf[:,1])
    #test_features.set_index('ticket_id', inplace=True)

    #  logistic regression- Gives AUC of 0.757789081252
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_curve, auc

    lr = LogisticRegression().fit(X_train, y_train)
    clf = lr.fit(X_train, y_train)
    y_score_lr = clf.decision_function(X_test)
    probabilities_lr = clf.predict_proba(test_features)

    test_features['compliance'] = pd.Series(probabilities_lr[:,1])
    test_features.set_index('ticket_id', inplace=True)

    
    return test_features.compliance

blight_model()


/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.75
285362    0.95
285361    0.80
285338    0.65
285346    0.80
285345    0.65
285347    0.88
285342    0.24
285530    0.80
284989    0.88
285344    0.80
285343    0.80
285340    0.79
285341    0.84
285349    0.81
285348    0.65
284991    0.88
285532    0.81
285406    0.93
285001    0.79
285006    0.90
285405    0.93
285337    0.89
285496    0.67
285497    0.60
285378    0.94
285589    0.92
285585    0.68
285501    0.65
285581    0.94
          ... 
376367    0.90
376366    0.87
376362    0.70
376363    0.59
376365    0.90
376364    0.87
376228    0.79
376265    0.81
376286    0.81
376320    0.85
376314    0.85
376327    0.61
376385    0.76
376435    0.65
376370    0.28
376434    0.83
376459    0.63
376478    0.94
376473    0.83
376484    0.87
376482    0.93
376480    0.95
376479    0.95
376481    0.95
376483    0.86
376496    0.98
376497    0.98
376499    0.69
376500    0.69
369851    0.58
Name: compliance, dtype: float64